In [44]:
# Navigate to appropriate directory
import os
while os.path.split(os.getcwd())[1] != "data-challenge-kernel-methods":
    os.chdir("..")
    if os.getcwd() == "/":
        raise ValueError()

import numpy as np
import kernels
import matplotlib.pyplot as plt
from PIL import Image
from time import time
from tqdm import tqdm
import pickle
import networkx as nx
import pandas as pd


from molecules import Molecule
from kernel_methods import KernelSVC, KernelPCA

In [2]:
with open("data/training_data.pkl", "rb") as file:
    training_data = pickle.load(file)

with open("data/training_labels.pkl", "rb") as file:
    training_labels = pickle.load(file)

with open("data/test_data.pkl", "rb") as file:
    test_data = pickle.load(file)

training_data = [Molecule(graph) for graph in training_data]
test_data = [Molecule(graph) for graph in test_data]

In [42]:
print(test_data)

[<molecules.Molecule object at 0x000001849388F850>, <molecules.Molecule object at 0x000001849388FC70>, <molecules.Molecule object at 0x00000184DB27A320>, <molecules.Molecule object at 0x00000184DB278610>, <molecules.Molecule object at 0x00000184DB278820>, <molecules.Molecule object at 0x00000184DB278580>, <molecules.Molecule object at 0x0000018493AFC3D0>, <molecules.Molecule object at 0x0000018493AFC790>, <molecules.Molecule object at 0x0000018493AFC9D0>, <molecules.Molecule object at 0x0000018493AFCCD0>, <molecules.Molecule object at 0x0000018493AFD090>, <molecules.Molecule object at 0x0000018493AFD2D0>, <molecules.Molecule object at 0x0000018493AFD5D0>, <molecules.Molecule object at 0x0000018493AFD990>, <molecules.Molecule object at 0x0000018493AFDCC0>, <molecules.Molecule object at 0x0000018493AFDF00>, <molecules.Molecule object at 0x0000018493AFE2C0>, <molecules.Molecule object at 0x0000018493AFE5C0>, <molecules.Molecule object at 0x0000018493AFE800>, <molecules.Molecule object at 

In [3]:
K = np.load('wlsk.npy')

In [4]:
K.shape

(6000, 6000)

In [5]:
training_labels.shape

(6000,)

In [6]:
#train a SVC on the training data with kernel matrix K and with svm of sklearn
#and compute the accuracy on the training data
from sklearn.svm import SVC
svc = SVC(kernel='precomputed',C=1)
svc.fit(K, training_labels)
svc.score(K, training_labels)



0.9938333333333333

In [17]:
idx = np.random.permutation(6000)

In [36]:
#train a SVC on the training data with kernel matrix K and with svm of sklearn with 

svc = SVC(kernel='precomputed',C=0.1)
svc.fit(K[idx[:5000],:][:,idx[:5000]], training_labels[idx[:5000]])
#with AUC as scoring metric
from sklearn.metrics import roc_auc_score
roc_auc_score(training_labels[idx[5000:]], svc.decision_function(K[idx[5000:],:][:,idx[:5000]]))


0.9095274831243972

In [37]:
print(training_labels.max())

1


In [38]:
K[idx[5000:],:][:,idx[:5000]].shape

(1000, 5000)

In [39]:
K_test = np.load('wlsk_test.npy')

In [40]:
K_test.shape

(6000, 2000)

In [41]:
#train SVC using the whole training data
svc = SVC(kernel='precomputed',C=0.1)
svc.fit(K, training_labels)
#plot score on training data
print(roc_auc_score(training_labels, svc.decision_function(K)))

#predict labels on test data of different size 
pred = svc.decision_function(K_test.T)




0.9896954806045715
[-1.10380762 -0.38985432 -1.33910107 ... -1.33944017  0.47873621
 -1.4926116 ]


In [46]:
Yte = {'Predicted' : pred} 
dataframe = pd.DataFrame(Yte) 
dataframe.index += 1 
dataframe.to_csv('test_pred.csv',index_label='Id') 